# Imports

In [2]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn
import pycaret.classification as pc

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


Missing colon in file WindowsPath('C:/Users/humberto.carvalho/Anaconda3/lib/site-packages/matplotlib/mpl-data/stylelib/gadfly.mplstyle'), line 7 ('Se você quiser manter uma cópia local permanente que não será')
Missing colon in file WindowsPath('C:/Users/humberto.carvalho/Anaconda3/lib/site-packages/matplotlib/mpl-data/stylelib/gadfly.mplstyle'), line 38 ('Se você omitir este parâmetro, o backend será determinado pelo recuo.')
Missing colon in file WindowsPath('C:/Users/humberto.carvalho/Anaconda3/lib/site-packages/matplotlib/mpl-data/stylelib/gadfly.mplstyle'), line 90 ('Os três padrões padrão de traço. Estes são dimensionados pela largura de linha.')

Bad key linhas.largura de linha in file C:\Users\humberto.carvalho\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\gadfly.mplstyle, line 76 ('linhas.largura de linha : 2 ## largura de linha em pontos')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.te

## Função create experiment

In [3]:
# Para usar o sqlite como repositorio
def create_experiment(experiment_name):
    mlflow.set_tracking_uri("sqlite:///mlruns_teste.db")
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        experiment = mlflow.get_experiment(experiment_id)
    return experiment.experiment_id

# 6- PreparacaoDados

In [4]:
def get_data():
    dir_file = '../data/dataset_kobe.csv'
    columns=['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_type', 'shot_made_flag']
    return pd.read_csv(dir_file, usecols=columns).query('shot_type == "2PT Field Goal"').dropna()

In [43]:
mlflow.set_tracking_uri("sqlite:///mlruns_teste_.db")
experiment_name = 'Preparacao Dados'
experiment_id = create_experiment(experiment_name)

with mlflow.start_run(experiment_id=experiment_id, run_name = experiment_name, nested=True):
    df = get_data()
    df.to_parquet("../data/processed/data_filtered.parquet")

    target_col = 'shot_made_flag'
    experiment = pc.setup(
        session_id=123,
        data = df, # Configurações de dados
        train_size=0.8,
        target = target_col,
        profile = False, # Analise interativa de variaveis
        fold_strategy = 'stratifiedkfold', # Validação cruzada
        fold = 10,
        normalize = True,  # Normalização, transformação e remoção de variáveis
        transformation = True,
        ignore_low_variance = True,
        remove_multicollinearity = True,
        multicollinearity_threshold = 0.95,
        bin_numeric_features = None, # Binarizacao de variaveis
        group_features = None, # Grupos de variáveis para combinar na engenharia de variaveis
        ignore_features = ['shot_type'],
        log_experiment = True, # Logging dos experimentos e afins
        experiment_name = experiment_name,
        silent=True
    )
    experiment[29][1][1].to_parquet("../data/operalization/base_train.parquet")
    experiment[29][3][1].to_parquet("../data/operalization/base_test.parquet")
    mlflow.log_artifact("../data/operalization/base_train.parquet")
    mlflow.log_artifact("../data/operalization/base_test.parquet")
df

,Description,Value
0,session_id,123
1,Target,shot_made_flag
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(20285, 8)"
5,Missing Values,0
6,Numeric Features,3
7,Categorical Features,3
8,Ordinal Features,0
9,High Cardinality Features,0


INFO:logs:Logging experiment in MLFlow
INFO:logs:SubProcess save_model() called ==================================
INFO:logs:Initializing save_model()
INFO:logs:save_model(model=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False,
                                      features_todrop=['shot_type'],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='shot_made_flag',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerica...
                ('dummy', Dummify(target='shot_made_flag')),
     

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag,shot_type
1,34.0443,-118.4268,10,1,0,15,0.0,2PT Field Goal
2,33.9093,-118.3708,7,1,0,16,1.0,2PT Field Goal
3,33.8693,-118.1318,6,1,0,22,0.0,2PT Field Goal
4,34.0443,-118.2698,6,2,0,0,1.0,2PT Field Goal
5,34.0553,-118.4148,9,3,0,14,0.0,2PT Field Goal
...,...,...,...,...,...,...,...,...
30690,33.9443,-118.3828,11,4,1,15,0.0,2PT Field Goal
30691,34.0443,-118.2698,7,4,1,0,0.0,2PT Field Goal
30692,33.9963,-118.2688,6,4,1,4,0.0,2PT Field Goal
30694,33.8783,-118.4038,3,4,1,21,1.0,2PT Field Goal


# 7- Treinamento

In [44]:
def get_final_model_train(experiment_name):
    index_train = pd.read_parquet("../data/operalization/base_train.parquet").index
    df_train = get_data().loc[index_train, :]
    target_col = 'shot_made_flag'
    experiment = pc.setup(
        session_id=123,
        data = df_train, # Configurações de dados
        train_size=0.8,
        target = target_col,
        profile = False, # Analise interativa de variaveis
        fold_strategy = 'stratifiedkfold', # Validação cruzada
        fold = 10,
        normalize = True,  # Normalização, transformação e remoção de variáveis
        transformation = True,
        ignore_low_variance = True,
        remove_multicollinearity = True,
        multicollinearity_threshold = 0.95,
        bin_numeric_features = None, # Binarizacao de variaveis
        group_features = None, # Grupos de variáveis para combinar na engenharia de variaveis
        ignore_features = ['shot_type'],
        log_experiment = True, # Logging dos experimentos e afins
        experiment_name = experiment_name,
        silent=True
    )

    model = pc.compare_models(n_select = 1, sort='Accuracy', include=['lr', 'rf'])
    tuned_model = pc.tune_model(model,
                                optimize = 'f1',
                                search_library = 'scikit-learn',
                                search_algorithm = 'random',
                                n_iter = 4)
    calibrated_model = pc.calibrate_model(tuned_model, method='sigmoid', calibrate_fold=5, fold=5)
    final_model = pc.finalize_model(calibrated_model)
    pred_final = pc.predict_model(final_model)
    pred_final.to_parquet('../data/operalization/pred_final.parquet')
    final_model_save = pc.save_model(calibrated_model, '../data/operalization/final_teste')
    return final_model, final_model_save

In [45]:
mlflow.set_tracking_uri("sqlite:///mlruns_teste_.db")
experiment_name = 'Treinamento'
model_name = 'modelo_salvo_final'
experiment_id = create_experiment(experiment_name)

with mlflow.start_run(experiment_id=experiment_id, run_name = experiment_name, nested=True) as run:
    final_model, final_model_save = get_final_model_train(experiment_name)
    classification_plots = ['log loss', 'f1', 'auc','pr','confusion_matrix', 'error', 'class_report', 'threshold', 'learning','vc','feature']
    for plot_type in classification_plots:
        print('=> Aplicando plot ', plot_type)
        try:
            artifact = pc.plot_model(final_model, plot=plot_type, save=True, use_train_data=False)
            mlflow.log_artifact(artifact)
        except:
            print('=> Nao possivel plotar: ', plot_type )
            continue

INFO:logs:Initializing plot_model()


INFO:logs:plot_model(fold=None, use_train_data=False, verbose=True, is_in_evaluate=False, display=None, display_format=None, estimator=CalibratedClassifierCV(base_estimator=LogisticRegression(C=8.866,
                                                         class_weight='balanced',
                                                         dual=False,
                                                         fit_intercept=True,
                                                         intercept_scaling=1,
                                                         l1_ratio=None,
                                                         max_iter=1000,
                                                         multi_class='auto',
                                                         n_jobs=None,
                                                         penalty='l2',
                                                         random_state=123,
                                                        

=> Nao possivel plotar:  vc
=> Aplicando plot  feature
=> Nao possivel plotar:  feature


In [48]:
experiment[0]

,shot_distance,minutes_remaining_0,minutes_remaining_1,minutes_remaining_10,minutes_remaining_11,minutes_remaining_2,minutes_remaining_3,minutes_remaining_4,minutes_remaining_5,minutes_remaining_6,...,minutes_remaining_8,minutes_remaining_9,period_1,period_2,period_3,period_4,period_5,period_6,period_7,playoffs_1
1,0.596373,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.743087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.663295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.311717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.451873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30690,0.596373,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
30691,-1.311717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
30692,-0.848414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
30694,1.505647,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# 8 - API Modelo

In [67]:
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
from mlflow.models.signature import ModelSignature
mlflow.set_registry_uri("sqlite:///mlruns_teste.db")

registered_model_name = 'modelo_teste'
model_version = -1
nexamples = 4

df_experiment = experiment[0]

model_features = list(df_experiment.columns)
inf_signature = infer_signature(df_experiment[model_features], final_model.predict(df_experiment))

input_example = {x: df_experiment[x].values[:nexamples] for x in model_features}

mlflow.sklearn.log_model(
    sk_model=final_model,
    artifact_path="sklearn-model",
    registered_model_name=registered_model_name,
    signature = inf_signature,
    input_example = input_example
)

client = MlflowClient()
if model_version == -1:
    model_version = client.get_latest_versions(registered_model_name)[-1].version

client.transition_model_version_stage(
    name=registered_model_name,
    version=model_version,
    stage="Staging"
)


2022/05/01 18:00:57 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\HUMBER~1.CAR\AppData\Local\Temp\tmpsu217vi_\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.23.2', 'cloudpickle==1.6.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'modelo_teste' already exists. Creating a new version of this model...
2022/05/01 18:00:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: modelo_teste, version 3
Created version '3' of model 'modelo_teste'.


<ModelVersion: creation_timestamp=1651438857483, current_stage='Staging', description=None, last_updated_timestamp=1651438857511, name='modelo_teste', run_id='542527e03c2742418f327f028edc7416', run_link=None, source='./mlruns/2/542527e03c2742418f327f028edc7416/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Servir o Modelo com o MLFlow

In [49]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns_teste.db'

!mlflow models serve -m "models:/modelo_teste/Staging" --no-conda -p 5001

^C


In [50]:
!mlflow server --backend-store-uri sqlite:///mlruns_teste.db --default-artifact-root file:./mlserver/

^C


## 8 - API REQUEST

In [55]:
dir_file = '../data/dataset_kobe.csv'
columns=['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_type', 'shot_made_flag']
df_predict = pd.read_csv(dir_file, usecols=columns).query('shot_type == "3PT Field Goal"').dropna()

In [56]:
import requests
host = 'localhost'
port = '5001'
url = f'http://{host}:{port}/invocations'
headers = {'Content-Type': 'application/json',}

http_data = df_predict.to_json(orient='split')
r = requests.post(url=url, headers=headers, data=http_data)

response = pd.read_json(r.text)
response.columns = ['shot_made_flag_api']
response.to_parquet('../data/processed/response_api.parquet')
response

,shot_made_flag_api
0,0
1,0
2,0
3,0
4,0
...,...
5407,0
5408,0
5409,0
5410,0


## f1_score

In [59]:
from sklearn.metrics import f1_score, log_loss

f1_score(df_predict['shot_made_flag'], response)

0.0

## Log_loss

In [61]:
log_loss(df_predict['shot_made_flag'], response)

11.385288005860222